In [2]:
import numpy as np

### Copper liquid rocket engine using methyl alcohol as the fuel propellant 

Target:
- F$\Delta$t = 1280 N*s

Propellant:
- Ethanol alcohol  + Oxygen Gas (O2)
- C2H5OH(l) + 2O2(g) → CO2(g) + 2H2O(l)

Assumes/Input parameters:

In [3]:
# Universal gas constant [J/K*mol]
Ru = 8.314 
# Molecular weight of Ethanol Alcohol [g/mol]
M = 22 
# Gravity [m/s^2]
g = 9.81
# gamma for ideal gas (Cp/Cv)
ga = 1.216
# Characteristic velocity[m/s]
cstar = 1645
# Mixture ratio (O/F) 
r = 1.3
# Thrust [N]  
F = 640
# Specific impulse [sec]
Isp = 228
# Chamber Pressure [Pa]
Pc = 1.5e6
# Specific gas constant of the propellant []
Rs = Ru/M 
# Chamber temperature [K]
Tc = 3030
# Atmosphere pressure [Pa]
Patm = 101350

In [4]:
# The total propellant flow rate [kg/s]
w = (F/Isp)/g

# Fuel flow rate
wf = w/(r+1)

# Oxygen flow rate
wo = w - wf

print(f'total mass flow rate={w:.3e} kg/s, fuel flow rate={wf:.3e} kg/s, oxidizer flow rate={wo:.3e} kg/s')
print(w,wf,wo)

total mass flow rate=2.861e-01 kg/s, fuel flow rate=1.244e-01 kg/s, oxidizer flow rate=1.617e-01 kg/s
0.2861383836758052 0.12440799290252401 0.16173039077328116


In [5]:
# Nozzel parameters
Tt = Tc * (1/(1+((ga-1)/2)))
                
Pt = Pc * ((1+((ga-1)/2))**-(ga/(ga-1)))

Cf = (Isp*g)/cstar

At = F/(Pc*Cf)

Dt = ((4*At)/3.14)**0.5

print(f'Throat Temperature={Tt:.3e} K, Throat Pessure={Pt:.3e} Pa, Throat Area={At:.3e} m^2, Throat Diameter={Dt:.3e} m')

Throat Temperature=2.735e+03 K, Throat Pessure=8.421e+05 Pa, Throat Area=3.138e-04 m^2, Throat Diameter=1.999e-02 m


In [6]:
# Exit parameters
Me = ((2/(ga-1))*(((Pc/Patm)**((ga-1)/ga))-1))**0.5

Ae = (1/Me)*((2/(ga+1))*(1+((ga-1)/2)*Me**2))**((ga+1)/(2*(ga-1)))*At

De = (4*Ae/np.pi)**0.5

print(f'Mach Number at the exit = {Me:.3f} ; Exit Area = {Ae:.3e} m^2 ; Exit Diameter = {De:.3e} m^2')

Mach Number at the exit = 2.384 ; Exit Area = 9.061e-04 m^2 ; Exit Diameter = 3.397e-02 m^2


In [7]:
# Assume the characteristic chamber length (L) is approx. 1.5 m , Dc = 5Dt , & Vconver = (1/10)Vc
L = 1.5
Dc = 3*Dt

Ac = ((Dc**2)*np.pi)/4

Vc = L*At

Lc = Vc/(1.1*Ac)

print(f'Combustion Chamber Surface Area={Ac:.3e} m^2, Combustion Chamber Volume={Vc:.3e} m^3, Length of Combustion Chamber={Lc:.3e} m')
print(Ac,Dc,Lc)

Combustion Chamber Surface Area=2.826e-03 m^2, Combustion Chamber Volume=4.707e-04 m^3, Length of Combustion Chamber=1.514e-01 m
0.0028256183149462082 0.05998073838256285 0.15143833979047014


In [15]:
# thickness of the throat and chamber
# Assume stress (s) is approximately 8000 psi or 55e6 Pa [1]
s = 55.15e6

tc = (Pc*Dc)/(2*s)

tc_actual = tc*13

print(f'Combustion Chamber Wall Thickness={tc_actual:.3e} m')
print(tc_actual)

Combustion Chamber Wall Thickness=1.060e-02 m
0.010604028997823894


#### Note about the wall thickness calculation
- the chamber and throat thickness calculation is the minimum thickness. The actual wall thickness should be greater to accommodate for the buckling force, welding, and stress concenstration [1,p18] 

In [10]:
# Average Heat Transfer Rate for using water
# Estimated Average Melting Point of Aluminum is around 640 C or 913 K [5]
# Aluminum Thermal Conductivity is around 251 W/m*K [5] and for Copper is 413 [6] 
#[W m−2]
q = 1.923e6 
# [K]
dTw = 278 
#[J kg−1 K−1]
cw = 4190 
mw= (1.1*np.pi*q*(Dc+2*tc)*Lc)/(cw*dTw)
vw = 9.2
rhow = 998.2
dgap = (((((4*mw)/(np.pi*vw*rhow))+(Dc+2*tc)**2)**0.5)-Dc-2*tc)/2
print(f'Mass flow rate of coolant water is = {mw:.3e} kg/s and annular coolant water flow gap is = {dgap:.3e} m')
print(mw)

Mass flow rate of coolant water is = 5.323e-02 kg/s and annular coolant water flow gap is = 2.993e-05 m
0.05323105433549041


In [41]:
# Injector for the fuel

# density of fuel at room temperature [kg/m^3]
RHO_fuel = 889

# volume flow rate is calculated as m^3/s
conversion_Lpermin = 60000
Vdot = (wf/RHO_fuel)*conversion_Lpermin

print(f'Volume flow rate of the fuel is {Vdot}L/min')

Volume flow rate of the fuel is 8.396489959675412L/min


In [11]:
# Injector for the oxidizer
Cd = 0.7
# Ambient temperature is assumed 20 C at atmospheric pressure [K]
Ta = 293.15
# Ambient pressure is assumed at atmospheric pressure [Pa]
Pa = Patm
# Density of oxidizer at ambient conditions
RHO_Oamb = Pa/(Ta*Rs)

# Pressure drop across the injector is commonly between 0.482*10^6 and 1.034*10^6 [Pa]
delta_Pinj = 1.034e6

# Pressure drop at the injector 
Pinj = Pc + delta_Pinj

# Temperature at the injector is assumed as room temperature[K]
Tinj = 293.15

# Density of oxidizer at the injector [kg/m^3]
RHO_Oinj = RHO_Oamb*(delta_Pinj/Pa)*(Ta/Tinj)

# Total area of the oxidizer injector [m^2]
Ao = wo/(Cd*(2*RHO_Oinj*delta_Pinj)**0.5)

# Area of oxidizer injector
Ao_inj = Ao

Do = (4*Ao_inj/np.pi)**0.5

print(f'Area of each oxidizer injection is ={Ao_inj:.3e} m^2, Diameter for each oxidizer injection is={Do:.3e} m')

Area of each oxidizer injection is =1.663e-06 m^2, Diameter for each oxidizer injection is=1.455e-03 m


### References:

- [1] L. J. Krzycki, HOW to DESIGN, BUILD and TEST SMALL LIQUID-FUEL ROCKET ENGINES. CA, USA: Rocketlab, 1967.
- [2] http://www.diva-portal.org/smash/get/diva2:1375729/FULLTEXT01.pdf
- [3] G. P. Sutton and O. Biblarz, Rocket Propulsion Elements, 9th ed. NJ, USA: John Wiley & Sons, Inc., 2017.